# Running State Arrays on SMALL-LABS formatted data

## Imports

In [3]:
from os.path import join
from pathlib import Path

from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib_scalebar.scalebar as sb
import numpy as np
from numpy.linalg import matrix_power
import pandas as pd
import scipy.io
from skimage.measure import find_contours, regionprops

from saspt import sample_detections, StateArray, StateArrayDataset, RBME
from saspt.lik import RBMELikelihood
import biteen_utilities as bu

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

### Find fits files

In [5]:
fits_folder = r"T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren"
fits_pattern = r"*_fits.mat"
fits_files = glob(join(fits_folder, r"*_fits.mat"))

print("{} fits files found!".format(len(fits_files)))

21 fits files found!


### Find phase masks (labels)

In [7]:
labels_folder = r"T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren"
labels_pattern = r"*_PhaseMask.mat"
labels_files = glob(join(labels_folder, labels_pattern))

print("{} labels files found!".format(len(labels_files)))

21 labels files found!


### Read fits and labels

In [8]:
fits_list = []
labels_list = []

max_track_id = 0
max_label_id = 0

for i, (ff, lf) in enumerate(zip(fits_files, labels_files)):
    fits_curr = bu.slfile_to_df(ff) # converts SMALL-LABS to pandas DataFrame
    fits_curr['movie #'] = i

    track_ids = fits_curr['track_id'].dropna().unique()
    n_tracks = len(track_ids)
    new_track_ids = np.arange(n_tracks) + max_track_id
    max_track_id = new_track_ids[-1] + 1
    fits_curr['track_id_unique'] = fits_curr['track_id'].replace(track_ids, new_track_ids)

    label_ids = fits_curr['roinum'].dropna().unique()
    n_labels = len(label_ids)
    new_label_ids = np.arange(n_labels) + max_label_id
    max_label_id = new_label_ids[-1] + 1
    fits_curr['roinum_unique'] = fits_curr['roinum'].replace(label_ids, new_label_ids)
    
    fits_list.append(fits_curr)
    labels_list.append(scipy.io.loadmat(lf)['PhaseMask'])

    print("Load data for {} and {}".format(ff, lf))

# Aggregate fits into single DataFrame
fits_all = pd.concat(fits_list, ignore_index=True)

Load data for T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_001_AccBGSUB_fits.mat and T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_001_PhaseMask.mat
Load data for T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_002_AccBGSUB_fits.mat and T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_002_PhaseMask.mat
Load data for T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_003_AccBGSUB_fits.mat and T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_003_PhaseMask.mat
Load data for T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_004_AccBGSUB_fits.mat and T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_004_PhaseMask.mat
Load data for T:\MIGRATED\Lab_Members\Daniel_Foust\Data\State Arrays\Lauren\230210_cwx2695_2_5hr_005_Acc

## Clean up data

### Exclude bad fits

In [9]:
fits_all = fits_all[fits_all['goodfit']==True]

### Exclude short tracks

In [10]:
fits_all = bu.filter_by_nlocs(fits_all,
                              min_locs=6,
                              max_locs=np.inf,
                              track_col='track_id')

## Calculate State Array

### Rename some columns
saspt expects columns with specific names. We change SMALL-LABS column labels to labels accepted by saspt.

In [14]:
fits_sa = fits_all.rename(columns={'col': 'x', 
                                   'row': 'y', 
                                   'track_id_unique': 'trajectory'})

### Define State Array

In [15]:
settings = dict(
    likelihood_type = RBME, # model that includes Brownian motion and localization error
    pixel_size_um = 0.049,  
    frame_interval = 0.04,  # in seconds
    focal_depth = np.inf,   # maybe should play around with this
    progress_bar = True,
)

SA = StateArray.from_detections(fits_sa[['x', 'y', 'trajectory', 'frame']], **settings)

# can ignore SettingWithCopyWarning

c:\Users\djfoust\Anaconda3\envs\saspt_env\lib\site-packages\saspt\trajectory_group.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detections[cls.DETECT_INDEX] = np.arange(len(detections), dtype=np.int64)


### Overwrite default likelihood model

In [19]:
rbme_likelihood = RBMELikelihood(
    pixel_size_um = 0.049,
    frame_interval = 0.04,
    focal_depth = np.inf,
    diff_coefs = np.logspace(-3, 1, 100), # adjust diffusion coefficients to consider here
    loc_errors = np.linspace(0, 0.1, 30)  # adjust localization errors to consider here
    )

SA.likelihood = rbme_likelihood

### Calculate State Arrays

In [20]:
SA.posterior_assignment_probabilities;

inferring posterior distribution...


100%|██████████| 200/200 [00:27<00:00,  7.32it/s]


#

In [21]:
track_posteriors = SA.posterior_assignment_probabilities.sum(axis=1)
Dav_track = (track_posteriors * SA.diff_coefs[:,None]).sum(axis=0)
sa_detections = SA.trajectories.detections
sa_detections['D_av'] = Dav_track[sa_detections['trajectory']]

posterior_occs = SA.posterior_occs.sum(axis=1)

In [22]:
posterior_occs.shape

(100,)

In [ ]:
all_fits['track_id_SA'] = np.nan
all_fits.loc[sa_detections['detect_index'], 'track_id_SA'] = sa_detections['trajectory']
all_fits.loc[sa_detections['detect_index'], 'D_av'] = sa_detections['D_av']

all_fits['log10(D_av)'] = np.log10(all_fits['D_av'])

In [ ]:
cmap = mpl.cm.cool

vmin = -3
vmax = 1
lmin = -2
lmax = 0

x = np.linspace(0, 1, 256)
n_min = len(x[x<(lmin-vmin) / (vmax - vmin)])
n_max = len(x[x>(lmax-vmin) / (vmax - vmin)])

seg1 = np.array(n_min * [cmap(0.),])
seg2 = cmap(np.linspace(0, 1, 256-n_min-n_max))
seg3 = np.array(n_max * [cmap(1.),])

colors = np.vstack((seg1, seg2, seg3))

coolramp = mpl.colors.ListedColormap(colors=np.vstack((seg1, seg2, seg3)), N=256, name='cool_sat')

In [ ]:
diff_coefs = SA.diff_coefs

figures_list = []

default_scalebar_props = {
    'dx': 0.049,
    'units': 'um',
    'fixed_value': 2,
    'scale_loc': 'none',
    'location': 'lower right',
    'frameon': False,
    'color': 'xkcd:white'
}

for imov in fits_all['movie #'].unique():
    movie_fits = all_fits[all_fits['movie #']==imov]
    movie_fits = movie_fits[~np.isnan(movie_fits['track_id_SA'])]
    movie_track_posteriors = track_posteriors[:, movie_fits['track_id_SA'].unique().astype('int')]

    movie_posterior_occs = (movie_track_posteriors * movie_fits.groupby('track_id_SA')['track_id_SA'].count().values).sum(axis=1) / len(movie_fits)

    phase_mask = phase_masks[imov]
    contours = bu.labels_to_contours(phase_mask, level=0.9)
    contours = [[smooth_polygon(contour[0])] for contour in contours]

    fig, ax = plt.subplots(2,1, figsize=(4,8))
    fig.suptitle(Path(files[imov]).stem, fontsize=12, y=0.92)
    ax[0].set_xscale('log')
    ax[0].set_xlim(left=diff_coefs[0], right=diff_coefs[-1])
    ax[0].set_ylim(bottom=0, top=movie_track_posteriors.max()*1.05)
    ax[0].plot(diff_coefs, movie_track_posteriors,
               color='xkcd:grey',
               lw=1.5,
               alpha=0.1)
    ax[0].plot(diff_coefs, movie_posterior_occs,
               color='xkcd:black',
               lw=3)
    ax[0].set_ylabel('Occupation', fontsize=14)
    
    ax[1].imshow(np.zeros(phase_mask.shape), cmap='binary_r')
    bu.crop_to_labels(ax[1], phase_mask)
    ax[1].set_axis_off()
    for contour in contours:
        ax[1].plot(contour[0][:,1], contour[0][:,0], lw=1, alpha=0.5, color='xkcd:gray')
    scatmap = ax[1].scatter(
        data=movie_fits.sort_values('D_av', ascending=False),
        # data=movie_fits,
        x='col',
        y='row',
        c='D_av',
        edgecolor='none',
        vmin=1e-3,
        vmax=1e1,
        alpha=0.33,
        s=5,
        cmap=coolramp,
        norm='log')
    ax[0].set_xticks([])
    cbar = plt.colorbar(scatmap,
                        orientation='horizontal',
                        ticks=[1e-3, 1e-2, 1e-1, 1e0, 1e1],
                        ax=ax[0],
                        label=r'$D_{app}\ (\mu m^2/s)$',
                        pad=0)
    
    cbar.solids.set(alpha=1)
    # cbar.set_label(fontsize=14)
    cbar.ax.set_xticklabels(['$\\mathdefault{10^{-3}}$',
                             '$\\mathdefault{10^{-2}}$',
                             '$\\mathdefault{10^{-1}}$',
                             '$\\mathdefault{10^{0}}$',
                             '$\\mathdefault{10^{1}}$'])
    # cbar.ax.tick_params(axis='x', labelsize=18)
    cbar.set_label(r'$D_{app}\ (\mu m^2/s)$', fontsize=14)
    
    scalebar = sb.ScaleBar(**default_scalebar_props)
    ax[1].add_artist(scalebar)
        
    all_figures.append(fig)